# Prompt Optimizer para sistema de recomendación (Goodreads)

Este notebook permite evaluar si un prompt es pertinente para un sistema de recomendación de libros de ficción y generar una versión mejorada del prompt para optimizar búsquedas sobre el dataset `goodreads_data.csv`.

**✨ Funcionalidad principal: BÚSQUEDA AUTOMÁTICA EN GOODREADS**
- Menciona un libro entre comillas (ej. "similar a '1984'")
- El sistema lo busca automáticamente en el dataset de Goodreads
- Extrae metadata real: autor, géneros, rating, valoraciones, año
- Genera prompt mejorado con datos reales del dataset
- **No necesitas ingresar manualmente ningún dato**

**Flujo**: 1) Configurar LLM (Gemini/OpenAI), 2) Cargar CSV, 3) Evaluar pertinencia, 4) Búsqueda automática + Generar prompt mejorado.

**Incluye modo `mock`** para pruebas sin consumir la API.

---
## ⚠️ Versión Corregida - Changelog:
- ✅ **Búsqueda automática de libros en dataset de Goodreads**
- ✅ **Extracción automática de metadata (autor, género, rating, etc.)**
- ✅ Carga segura de API key (sin hardcodear)
- ✅ Implementación de función `generate_text()`
- ✅ Carga automática del dataset CSV
- ✅ Corrección de regex para extracción de JSON
- ✅ Mejora de la función `improve_prompt()` con análisis inteligente
- ✅ Eliminación de documentación duplicada
- ✅ Manejo robusto de errores con logging
- ✅ Validación del dataset


## 1. Verificación de dependencias

Verifica que los paquetes necesarios estén instalados. Si falta alguno, instálalo con `uv`.

In [21]:
# Verificar dependencias instaladas
import sys

def check_package(package_name, import_name=None):
    """Verifica si un paquete está disponible"""
    import_name = import_name or package_name
    try:
        __import__(import_name)
        print(f"✓ {package_name} está instalado")
        return True
    except ImportError:
        print(f"✗ {package_name} NO está instalado")
        return False

print("Verificando dependencias necesarias:\n")

# Verificar paquetes básicos
check_package('pandas')
check_package('python-dotenv', 'dotenv')

# Verificar paquetes opcionales según proveedor
print("\nPaquetes opcionales (según proveedor LLM):")
check_package('google-generativeai', 'google.generativeai')
check_package('openai')

print("\n" + "="*60)
print("📦 Si falta algún paquete, instálalo con uv:")
print("   uv add pandas python-dotenv")
print("   uv add google-generativeai  # para Gemini")
print("   uv add openai               # para OpenAI")
print("="*60)


Verificando dependencias necesarias:

✓ pandas está instalado
✓ python-dotenv está instalado

Paquetes opcionales (según proveedor LLM):
✓ google-generativeai está instalado
✓ openai está instalado

📦 Si falta algún paquete, instálalo con uv:
   uv add pandas python-dotenv
   uv add google-generativeai  # para Gemini
   uv add openai               # para OpenAI


## 2. Imports y configuración inicial

In [22]:
# Imports principales
import os
import json
import re
import logging
from pathlib import Path
from getpass import getpass
import pandas as pd

# Configurar logging para debugging
logging.basicConfig(
    level=logging.INFO,
    format='%(levelname)s: %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ Imports completados")

✅ Imports completados


## 3. Configuración del Modelo LLM y API Key

Configura el proveedor de LLM (`gemini` o `openai`) y carga la API key de forma segura.

**Opciones para cargar API key**:
1. Variable de entorno (recomendado): `export GEMINI_API_KEY=tu_clave`
2. Archivo externo: `api_key_gemini.txt` (no debe estar en git)
3. Input manual del usuario (getpass)

In [23]:
# Configuración del modelo
MODEL_NAME = 'models/gemini-2.0-flash-exp'  # o 'gpt-4o' para OpenAI
provider = 'gemini'  # 'gemini' o 'openai'

# Carga segura de API key con soporte para .env
API_KEY = None

# Intentar cargar variables de entorno desde archivo .env
try:
    from dotenv import load_dotenv
    if load_dotenv():  # Carga .env si existe
        logger.info("✓ Archivo .env encontrado y cargado")
except ImportError:
    logger.debug("python-dotenv no instalado, continuando...")

# Opción 1: Cargar desde variable de entorno (incluye .env si se cargó)
env_var_name = 'GEMINI_API_KEY' if provider == 'gemini' else 'OPENAI_API_KEY'
API_KEY = os.environ.get(env_var_name)
if API_KEY:
    logger.info(f"✓ API key cargada desde variable de entorno {env_var_name}")

# Opción 2: Cargar desde archivo de texto (si existe)
if not API_KEY:
    api_key_file = Path('api_key_gemini.txt') if provider == 'gemini' else Path('api_key_openai.txt')
    if api_key_file.exists():
        with open(api_key_file, 'r') as f:
            API_KEY = f.read().strip()
        logger.info(f"✓ API key cargada desde archivo {api_key_file}")

# Opción 3: Solicitar al usuario
if not API_KEY:
    API_KEY = getpass(f'Ingresa tu API key para {provider} (entrada oculta): ')
    logger.info("✓ API key ingresada manualmente")

if not API_KEY:
    logger.warning("⚠️ No se configuró API key. Solo funcionará el modo mock.")
else:
    logger.info(f"✅ Configuración completa: {provider} - {MODEL_NAME}")

print(f"\nModelo: {MODEL_NAME}")
print(f"Proveedor: {provider}")
print(f"API Key configurada: {'Sí ✓' if API_KEY else 'No (solo modo mock)'}")


INFO: ✓ Archivo .env encontrado y cargado
INFO: ✓ API key cargada desde variable de entorno GEMINI_API_KEY
INFO: ✅ Configuración completa: gemini - models/gemini-2.0-flash-exp
INFO: ✓ API key cargada desde variable de entorno GEMINI_API_KEY
INFO: ✅ Configuración completa: gemini - models/gemini-2.0-flash-exp



Modelo: models/gemini-2.0-flash-exp
Proveedor: gemini
API Key configurada: Sí ✓


## 4. Cargar Dataset de Goodreads

Carga el archivo CSV y valida su estructura.

In [24]:
def load_and_validate_dataset(csv_path='goodreads_data.csv'):
    """Carga y valida el dataset de Goodreads"""
    try:
        df = pd.read_csv(csv_path)
        logger.info(f"✓ Dataset cargado: {len(df)} registros, {len(df.columns)} columnas")
        
        # Validar columnas esperadas
        expected_cols = ['Book', 'Author', 'Genres', 'Description']
        missing = [col for col in expected_cols if col not in df.columns]
        
        if missing:
            logger.warning(f"⚠️ Columnas esperadas pero no encontradas: {missing}")
        else:
            logger.info("✓ Todas las columnas esperadas están presentes")
        
        # Mostrar información básica
        print(f"\n{'='*60}")
        print("INFORMACIÓN DEL DATASET")
        print(f"{'='*60}")
        print(f"Registros: {len(df):,}")
        print(f"Columnas: {', '.join(df.columns.tolist())}")
        print(f"\nPrimeros registros:")
        print(df.head(3)[['Book', 'Author', 'Avg_Rating']].to_string())
        print(f"{'='*60}\n")
        
        return df
    
    except FileNotFoundError:
        logger.error(f"❌ Archivo no encontrado: {csv_path}")
        logger.info("Intentando buscar en subdirectorios...")
        # Buscar en subdirectorios comunes
        for path in ['./TP03/goodreads_data.csv', './TPv02/goodreads_data.csv']:
            if Path(path).exists():
                logger.info(f"✓ Encontrado en: {path}")
                return load_and_validate_dataset(path)
        logger.error("❌ No se pudo encontrar goodreads_data.csv")
        return None
    
    except Exception as e:
        logger.error(f"❌ Error al cargar dataset: {e}")
        return None

# Cargar el dataset
df = load_and_validate_dataset()

if df is None:
    print("\n⚠️ ADVERTENCIA: Dataset no cargado. Las funciones estarán limitadas.")

INFO: ✓ Dataset cargado: 10000 registros, 8 columnas
INFO: ✓ Todas las columnas esperadas están presentes
INFO: ✓ Todas las columnas esperadas están presentes



INFORMACIÓN DEL DATASET
Registros: 10,000
Columnas: Unnamed: 0, Book, Author, Description, Genres, Avg_Rating, Num_Ratings, URL

Primeros registros:
                                                          Book        Author  Avg_Rating
0                                        To Kill a Mockingbird    Harper Lee        4.27
1  Harry Potter and the Philosopher’s Stone (Harry Potter, #1)  J.K. Rowling        4.47
2                                          Pride and Prejudice   Jane Austen        4.28



## 5. Implementación de la Función de Generación de Texto (LLM)

Esta función conecta con la API del proveedor seleccionado (Gemini o OpenAI).

In [25]:
def generate_text(prompt, mock=False, temperature=0.7, max_tokens=500):
    """
    Genera texto usando el LLM configurado.
    
    Args:
        prompt (str): El prompt a enviar al LLM
        mock (bool): Si True, devuelve respuesta simulada
        temperature (float): Temperatura para la generación (0-1)
        max_tokens (int): Máximo de tokens a generar
    
    Returns:
        str: Respuesta del LLM o respuesta mock
    """
    if mock:
        logger.info("🎭 Modo MOCK activado - generando respuesta simulada")
        # Respuestas simuladas según el tipo de prompt
        if "is_relevant" in prompt.lower():
            return json.dumps({
                "is_relevant": "yes",
                "explanation": "El prompt menciona recomendación de libros, lo cual es relevante para el sistema."
            })
        else:
            return json.dumps({
                "improved_prompt": "Recomiéndame 10 libros del género fantasía contemporánea, publicados después de 2010, con rating promedio superior a 4.0 y al menos 1000 valoraciones.",
                "notes": [
                    "Se añadió especificación de cantidad (10 libros)",
                    "Se incluyó filtro temporal (después de 2010)",
                    "Se agregaron filtros de calidad (rating > 4.0, votos > 1000)"
                ]
            })
    
    if not API_KEY:
        raise ValueError("API_KEY no configurada. Usa mock=True para modo de prueba.")
    
    try:
        if provider == 'gemini':
            import google.generativeai as genai
            
            genai.configure(api_key=API_KEY)
            model = genai.GenerativeModel(MODEL_NAME)
            
            generation_config = {
                "temperature": temperature,
                "max_output_tokens": max_tokens,
            }
            
            logger.info(f"📡 Llamando a Gemini ({MODEL_NAME})...")
            response = model.generate_content(
                prompt,
                generation_config=generation_config
            )
            
            result = response.text
            logger.info(f"✓ Respuesta recibida ({len(result)} caracteres)")
            return result
        
        elif provider == 'openai':
            import openai
            
            openai.api_key = API_KEY
            
            logger.info(f"📡 Llamando a OpenAI ({MODEL_NAME})...")
            response = openai.ChatCompletion.create(
                model=MODEL_NAME,
                messages=[
                    {"role": "system", "content": "Eres un asistente experto en sistemas de recomendación de libros."},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
                max_tokens=max_tokens
            )
            
            result = response.choices[0].message.content
            logger.info(f"✓ Respuesta recibida ({len(result)} caracteres)")
            return result
        
        else:
            raise ValueError(f"Proveedor no soportado: {provider}")
    
    except Exception as e:
        logger.error(f"❌ Error al generar texto: {e}")
        raise

print("✅ Función generate_text() implementada")

✅ Función generate_text() implementada


## 6. Heurística Local para Evaluar Pertinencia

Evaluación rápida (sin consumir API) basada en palabras clave.

In [26]:
RELEVANT_KEYWORDS = [
    'recommend', 'recomendar', 'sugerir', 'suggest',
    'book', 'libro', 'novel', 'novela',
    'fiction', 'ficción',
    'recommendation', 'recomendación',
    'genre', 'género', 'genero',
    'find', 'buscar', 'search',
    'similar', 'parecido', 'like',
    'read', 'leer', 'reading'
]

def heuristic_relevance_check(prompt):
    """
    Evaluación heurística rápida de pertinencia.
    
    Returns:
        tuple: (is_relevant: bool, score: int)
    """
    p = prompt.lower()
    score = sum(1 for kw in RELEVANT_KEYWORDS if kw in p)
    is_relevant = score >= 1
    
    logger.debug(f"Heurística: score={score}, relevante={is_relevant}")
    return is_relevant, score


def extract_json_from_text(text):
    """
    Extrae el primer objeto JSON válido del texto.
    Corrección del regex defectuoso en la versión original.
    """
    # Intentar con regex simple primero
    patterns = [
        r'\{[^{}]*\}',  # JSON simple de una línea
        r'\{.*?\}',      # JSON multilínea (non-greedy)
    ]
    
    for pattern in patterns:
        matches = re.finditer(pattern, text, flags=re.DOTALL)
        for match in matches:
            try:
                return json.loads(match.group(0))
            except json.JSONDecodeError:
                continue
    
    # Si no se pudo extraer JSON, crear objeto básico
    logger.debug("No se pudo extraer JSON válido, creando objeto básico")
    if 'yes' in text.lower() or 'sí' in text.lower():
        return {'is_relevant': 'yes', 'explanation': text.strip()}
    else:
        return {'is_relevant': 'no', 'explanation': text.strip()}


def analyze_prompt_relevance(prompt, df=None, use_llm=True, mock=False):
    """
    Analiza si un prompt es relevante para el sistema de recomendación.
    
    Returns:
        tuple: (is_relevant: bool, reason: str, llm_output: str or None)
    """
    logger.info(f"Analizando pertinencia del prompt: '{prompt[:50]}...'")
    
    # Heurística rápida
    relevance, score = heuristic_relevance_check(prompt)
    reason_parts = [f'heuristic_score={score}']
    llm_out = None
    
    if use_llm:
        # Construir prompt para LLM
        llm_prompt = f"""Eres un clasificador experto. Analiza si el siguiente prompt es relevante para un sistema de recomendación de libros de ficción:

PROMPT DEL USUARIO:
{prompt}

Responde ÚNICAMENTE en formato JSON con estos campos:
{{
  "is_relevant": "yes" o "no",
  "explanation": "1-2 frases explicando tu decisión"
}}
"""
        
        try:
            llm_out = generate_text(llm_prompt, mock=mock)
            j = extract_json_from_text(llm_out)
            
            is_rel = str(j.get('is_relevant', 'no')).lower().startswith('y')
            reason_parts.append('llm_eval=' + ('yes' if is_rel else 'no'))
            
            explanation = j.get('explanation', '')
            logger.info(f"LLM evaluación: {'relevante' if is_rel else 'no relevante'}")
            
            return is_rel, '; '.join(reason_parts + [explanation]), llm_out
        
        except Exception as e:
            logger.error(f"Error en evaluación LLM: {e}")
            reason_parts.append(f'llm_error={type(e).__name__}')
    
    return relevance, '; '.join(reason_parts), llm_out

print("✅ Funciones de evaluación de pertinencia implementadas")

✅ Funciones de evaluación de pertinencia implementadas


## 7. Mejora Inteligente del Prompt

La función `improve_prompt` **busca automáticamente** los libros mencionados en el dataset de Goodreads y extrae sus datos:

**Funcionalidad automática:**
- 🔍 Detecta libros mencionados en el prompt (ej. "similar a '1984'")
- 📚 Busca el libro en el dataset de Goodreads
- 📊 **Extrae automáticamente**: autor, géneros, rating, número de valoraciones, año
- ✨ Genera prompt mejorado con filtros basados en datos reales del dataset

**No necesitas ingresar manualmente** género, autor, etc. - el sistema los obtiene automáticamente del dataset.

In [33]:
def summarize_df_columns(df, max_cols=15):
    """Resume las columnas del dataframe."""
    if df is None:
        return 'Dataset no cargado'
    cols = list(df.columns)[:max_cols]
    result = ', '.join(cols)
    if len(df.columns) > max_cols:
        result += f' ... (+{len(df.columns) - max_cols} más)'
    return result


def extract_book_reference(prompt):
    """Extrae menciones a libros específicos del prompt."""
    # Patrones que aceptan comillas rectas (' ") y comillas tipográficas (Unicode)
    # Usando códigos Unicode explícitos para evitar problemas de sintaxis
    patterns = [
        r"similar(?:\s+a)?\s+['\"\u2018\u2019\u201C\u201D]([^'\"\u2018\u2019\u201C\u201D]+)['\"\u2018\u2019\u201C\u201D]",  # similar a "titulo"
        r"como\s+['\"\u2018\u2019\u201C\u201D]([^'\"\u2018\u2019\u201C\u201D]+)['\"\u2018\u2019\u201C\u201D]",  # como "titulo"
        r"parecido\s+a\s+['\"\u2018\u2019\u201C\u201D]([^'\"\u2018\u2019\u201C\u201D]+)['\"\u2018\u2019\u201C\u201D]",  # parecido a "titulo"
        r"['\"\u2018\u2019\u201C\u201D]([^'\"\u2018\u2019\u201C\u201D]{3,})['\"\u2018\u2019\u201C\u201D]",  # cualquier "Titulo" (mínimo 3 caracteres)
    ]
    
    for pat in patterns:
        m = re.search(pat, prompt, flags=re.IGNORECASE)
        if m:
            candidate = m.group(1).strip()
            if len(candidate) > 2:
                logger.debug(f"Libro detectado: '{candidate}'")
                return candidate
    
    return None


def search_book_in_df(df, book_title):
    """Busca un libro en el dataframe."""
    if df is None or book_title is None:
        return None
    
    # Buscar columna de título
    title_col = None
    for col in df.columns:
        if col.lower() in ['title', 'book', 'name', 'titulo']:
            title_col = col
            break
    
    if title_col is None:
        logger.warning("No se encontró columna de título en el dataset")
        return None
    
    # Búsqueda case-insensitive
    bt_lower = book_title.lower()
    matches = df[df[title_col].astype(str).str.lower().str.contains(bt_lower, na=False, regex=False)]
    
    if len(matches) > 0:
        logger.info(f"✓ Libro encontrado: {matches.iloc[0][title_col]}")
        return matches.iloc[0]
    
    logger.debug(f"Libro '{book_title}' no encontrado en el dataset")
    return None


def improve_prompt(prompt, df=None, use_llm=False, mock=False):
    """
    Mejora el prompt usando análisis inteligente del dataset.
    
    Estrategia:
    1. Detectar si menciona un libro específico
    2. Buscar ese libro en el dataset de Goodreads
    3. Extraer metadata automáticamente (género, autor, rating, etc.)
    4. Construir prompt mejorado con filtros específicos basados en datos reales
    
    Returns:
        tuple: (improved_prompt: str, notes: list)
    """
    logger.info("Mejorando prompt...")
    notes = []
    
    # Información sobre columnas disponibles
    cols_summary = summarize_df_columns(df)
    notes.append(f'Columnas disponibles: {cols_summary}')
    
    # Intentar detectar libro de referencia
    book_ref = extract_book_reference(prompt)
    
    if book_ref and df is not None:
        notes.append(f'📚 Libro mencionado en el prompt: "{book_ref}"')
        
        # Buscar en el dataset de Goodreads
        found_row = search_book_in_df(df, book_ref)
        
        if found_row is not None:
            # ✅ LIBRO ENCONTRADO - Extraer datos automáticamente del dataset
            notes.append(f'✅ Libro encontrado en el dataset de Goodreads')
            improved_parts = []
            metadata_info = []
            
            # Título exacto
            if 'Book' in found_row:
                title = found_row['Book']
                improved_parts.append(f'Recomendar libros similares a "{title}"')
                metadata_info.append(f'📖 Título: {title}')
            
            # Autor (extraído del dataset)
            if 'Author' in found_row and pd.notna(found_row['Author']):
                author = found_row['Author']
                metadata_info.append(f'✍️ Autor: {author}')
                notes.append(f'Autor del libro de referencia: {author}')
            
            # Géneros (extraídos del dataset)
            if 'Genres' in found_row and pd.notna(found_row['Genres']):
                genres = str(found_row['Genres'])
                improved_parts.append(f'del género: {genres[:100]}')
                metadata_info.append(f'🎭 Géneros: {genres[:80]}...' if len(genres) > 80 else f'🎭 Géneros: {genres}')
                notes.append(f'Géneros automáticamente detectados: {genres[:60]}...' if len(genres) > 60 else f'Géneros: {genres}')
            
            # Rating (extraído del dataset)
            if 'Avg_Rating' in found_row and pd.notna(found_row['Avg_Rating']):
                rating = float(found_row['Avg_Rating'])
                min_rating = max(3.5, rating - 0.5)
                improved_parts.append(f'con rating mínimo: {min_rating:.1f}')
                metadata_info.append(f'⭐ Rating del libro de referencia: {rating:.2f}')
                notes.append(f'Rating automáticamente detectado: {rating:.2f} → filtro sugerido: mínimo {min_rating:.1f}')
            
            # Número de valoraciones (extraído del dataset)
            if 'Num_Ratings' in found_row and pd.notna(found_row['Num_Ratings']):
                num_ratings_raw = found_row['Num_Ratings']
                num_ratings_converted = pd.to_numeric(num_ratings_raw, errors='coerce')
                if pd.notna(num_ratings_converted):
                    num_ratings = int(num_ratings_converted)
                    min_votes = max(100, num_ratings // 20)
                    improved_parts.append(f'y al menos {min_votes:,} valoraciones')
                    metadata_info.append(f'👥 Valoraciones del libro de referencia: {num_ratings:,}')
                    notes.append(f'Valoraciones automáticamente detectadas: {num_ratings:,} → filtro sugerido: mínimo {min_votes:,}')
            
            # Año de publicación (si está disponible)
            if 'Year' in found_row and pd.notna(found_row['Year']):
                year = int(found_row['Year'])
                metadata_info.append(f'📅 Año: {year}')
                notes.append(f'Año de publicación: {year}')
            
            improved_parts.append('| Limitar resultados a top 10')
            
            improved = ' '.join(improved_parts)
            
            # Agregar resumen de metadata extraída
            notes.append('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')
            notes.append('📊 METADATA EXTRAÍDA DEL DATASET DE GOODREADS:')
            notes.extend(metadata_info)
            notes.append('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')
            notes.append('✅ Prompt mejorado automáticamente con datos reales del dataset')
        
        else:
            # ❌ Libro no encontrado en el dataset
            improved = f'{prompt} | Nota: El libro "{book_ref}" no se encontró en el dataset de Goodreads. Usar filtros genéricos: género específico, rating>4.0, votos>500, limitar a 10 resultados.'
            notes.append(f'⚠️ Libro "{book_ref}" NO encontrado en el dataset')
            notes.append(f'💡 Sugerencia: Verifica el título exacto o prueba con otro libro')
    
    else:
        # No se detectó libro específico: mejora genérica
        improved = f"{prompt} | Sugerencias: especificar un libro de ejemplo (ej. 'similar a \"1984\"'), género, rango de rating (ej. >4.0), votos mínimos (ej. >500), número de resultados (ej. top 10)."
        notes.append('ℹ️ No se detectó mención a un libro específico en el prompt')
        notes.append('💡 Sugerencia: Menciona un libro entre comillas para búsqueda automática en Goodreads')
    
    logger.info("✓ Prompt mejorado generado")
    return improved, notes

print("✅ Funciones de mejora de prompt implementadas")
print("💡 El sistema ahora busca automáticamente libros en el dataset de Goodreads")
print("   y extrae: título, autor, géneros, rating, valoraciones, año")
print("🔧 ACTUALIZADO: Ahora detecta comillas tipográficas (Unicode) y comillas rectas")
print("   Soporta: comillas simples rectas ('), dobles rectas (\"), y comillas tipográficas")


✅ Funciones de mejora de prompt implementadas
💡 El sistema ahora busca automáticamente libros en el dataset de Goodreads
   y extrae: título, autor, géneros, rating, valoraciones, año
🔧 ACTUALIZADO: Ahora detecta comillas tipográficas (Unicode) y comillas rectas
   Soporta: comillas simples rectas ('), dobles rectas ("), y comillas tipográficas


## 8. Uso: Evaluar y Mejorar un Prompt

Define tu prompt en la variable `PROMPT` y ejecuta para ver:
- ✅ Evaluación de pertinencia
- 🔍 **Búsqueda automática** del libro mencionado en Goodreads
- 📊 **Extracción automática** de metadata (autor, género, rating, etc.)
- ✨ Versión mejorada del prompt con datos reales
- 📝 Notas explicativas detalladas

**Cómo funciona la búsqueda automática:**
1. Menciona un libro entre comillas en tu prompt: `"similar a '1984'"`
2. El sistema lo busca automáticamente en el dataset de Goodreads
3. Extrae: autor, géneros, rating, valoraciones, año
4. Genera un prompt mejorado con esos datos reales

**No necesitas ingresar manualmente** ningún dato - el sistema los encuentra automáticamente.

In [34]:
# ========================================
# CONFIGURACIÓN - Edita aquí tu prompt
# ========================================

# Simplemente menciona un libro entre comillas y el sistema buscará automáticamente
# todos sus datos (autor, género, rating, etc.) en el dataset de Goodreads
PROMPT = "Recomiéndame libros similares a '1984'"

# Opciones de ejecución
use_llm = False   # True para usar LLM real (requiere API_KEY configurada)
mock = True       # True para modo de prueba sin consumir API

# ========================================
# EJECUCIÓN
# ========================================

print("\n" + "="*70)
print("ANÁLISIS DE PROMPT")
print("="*70)
print(f"\nPrompt original:\n  \"{PROMPT}\"\n")
print("💡 El sistema buscará automáticamente '1984' en Goodreads")
print("   y extraerá: autor, géneros, rating, valoraciones, año\n")

# 1. Evaluar pertinencia
print("🔍 Evaluando pertinencia...")
is_rel, reason, llm_out = analyze_prompt_relevance(
    PROMPT, 
    df=df if df is not None else None, 
    use_llm=use_llm, 
    mock=mock
)

print(f"\n{'='*70}")
print("RESULTADO DE PERTINENCIA")
print(f"{'='*70}")
print(f"  ¿Es pertinente?: {'✅ SÍ' if is_rel else '❌ NO'}")
print(f"  Razón: {reason}")

if llm_out and use_llm:
    print(f"\n  Respuesta del LLM:\n  {llm_out}")

# 2. Mejorar prompt (búsqueda automática en Goodreads)
print(f"\n{'='*70}")
print("🔧 Mejorando prompt con búsqueda automática en Goodreads...")
improved, notes = improve_prompt(
    PROMPT, 
    df=df if df is not None else None, 
    use_llm=use_llm, 
    mock=mock
)

print(f"\n{'='*70}")
print("PROMPT MEJORADO")
print(f"{'='*70}")
print(f"\n{improved}\n")

print(f"{'='*70}")
print("METADATA EXTRAÍDA Y NOTAS")
print(f"{'='*70}")
for i, note in enumerate(notes, 1):
    print(f"  {note}")

print(f"\n{'='*70}\n")


INFO: Analizando pertinencia del prompt: 'Recomiéndame libros similares a '1984'...'
INFO: Mejorando prompt...
INFO: Mejorando prompt...
INFO: ✓ Libro encontrado: 1984
INFO: ✓ Prompt mejorado generado
INFO: ✓ Libro encontrado: 1984
INFO: ✓ Prompt mejorado generado



ANÁLISIS DE PROMPT

Prompt original:
  "Recomiéndame libros similares a '1984'"

💡 El sistema buscará automáticamente '1984' en Goodreads
   y extraerá: autor, géneros, rating, valoraciones, año

🔍 Evaluando pertinencia...

RESULTADO DE PERTINENCIA
  ¿Es pertinente?: ✅ SÍ
  Razón: heuristic_score=2

🔧 Mejorando prompt con búsqueda automática en Goodreads...

PROMPT MEJORADO

Recomendar libros similares a "1984" del género: ['Classics', 'Fiction', 'Science Fiction', 'Dystopia', 'Literature', 'Novels', 'Politics'] con rating mínimo: 3.7 | Limitar resultados a top 10

METADATA EXTRAÍDA Y NOTAS
  Columnas disponibles: Unnamed: 0, Book, Author, Description, Genres, Avg_Rating, Num_Ratings, URL
  📚 Libro mencionado en el prompt: "1984"
  ✅ Libro encontrado en el dataset de Goodreads
  Autor del libro de referencia: George Orwell
  Géneros automáticamente detectados: ['Classics', 'Fiction', 'Science Fiction', 'Dystopia', 'Lite...
  Rating automáticamente detectado: 4.19 → filtro sugerido: mí

In [35]:
Stop 

NameError: name 'Stop' is not defined

## 9. Ejemplos Adicionales

Prueba con diferentes tipos de prompts. Los que mencionan libros específicos entre comillas activarán la **búsqueda automática** en Goodreads.

In [ ]:
# Ejemplos de prompts para probar
# Los que tienen libros entre comillas activarán búsqueda automática en Goodreads
ejemplos = [
    "libros de fantasía",  # Sin libro específico - mejora genérica
    "algo como 'Harry Potter'",  # CON libro - busca automáticamente en Goodreads
    "similar a 'The Hunger Games'",  # CON libro - extrae metadata automáticamente
    "quiero leer ciencia ficción distópica",  # Sin libro específico
    "recomiéndame algo parecido a 'To Kill a Mockingbird'",  # CON libro - búsqueda automática
]

print("\n" + "="*70)
print("EJEMPLOS DE MEJORA DE PROMPTS")
print("="*70)
print("\n💡 Los prompts con libros entre comillas activarán búsqueda automática")
print("   en el dataset de Goodreads para extraer metadata real\n")

for i, ejemplo in enumerate(ejemplos, 1):
    print(f"\n{i}. ORIGINAL: \"{ejemplo}\"")
    
    # Evaluar
    is_rel, _, _ = analyze_prompt_relevance(ejemplo, df=df, use_llm=False, mock=True)
    print(f"   Pertinente: {'✅' if is_rel else '❌'}")
    
    # Mejorar (con búsqueda automática si menciona libro)
    improved, notes = improve_prompt(ejemplo, df=df, use_llm=False, mock=True)
    print(f"   MEJORADO: {improved[:180]}...")
    
    # Mostrar si encontró libro automáticamente
    if any('encontrado en el dataset' in note for note in notes):
        print(f"   🎯 Libro encontrado - metadata extraída automáticamente")
    elif any('mencionado en el prompt' in note for note in notes):
        print(f"   ⚠️ Libro mencionado pero no encontrado en dataset")
    print()

print("="*70)


INFO: Analizando pertinencia del prompt: 'libros de fantasía...'
INFO: Mejorando prompt...
INFO: ✓ Prompt mejorado generado
INFO: Analizando pertinencia del prompt: 'algo como 'Harry Potter'...'
INFO: Mejorando prompt...
INFO: ✓ Libro encontrado: Harry Potter and the Philosopher’s Stone (Harry Potter, #1)
INFO: ✓ Prompt mejorado generado
INFO: Analizando pertinencia del prompt: 'similar a 'The Hunger Games'...'
INFO: Mejorando prompt...
INFO: ✓ Libro encontrado: The Hunger Games (The Hunger Games, #1)
INFO: ✓ Prompt mejorado generado
INFO: Mejorando prompt...
INFO: ✓ Prompt mejorado generado
INFO: Analizando pertinencia del prompt: 'algo como 'Harry Potter'...'
INFO: Mejorando prompt...
INFO: ✓ Libro encontrado: Harry Potter and the Philosopher’s Stone (Harry Potter, #1)
INFO: ✓ Prompt mejorado generado
INFO: Analizando pertinencia del prompt: 'similar a 'The Hunger Games'...'
INFO: Mejorando prompt...
INFO: ✓ Libro encontrado: The Hunger Games (The Hunger Games, #1)
INFO: ✓ Prompt mej


EJEMPLOS DE MEJORA DE PROMPTS

💡 Los prompts con libros entre comillas activarán búsqueda automática
   en el dataset de Goodreads para extraer metadata real


1. ORIGINAL: "libros de fantasía"
   Pertinente: ✅
   MEJORADO: libros de fantasía | Sugerencias: especificar un libro de ejemplo (ej. 'similar a "1984"'), género, rango de rating (ej. >4.0), votos mínimos (ej. >500), número de resultados (ej. ...


2. ORIGINAL: "algo como 'Harry Potter'"
   Pertinente: ❌
   MEJORADO: Recomendar libros similares a "Harry Potter and the Philosopher’s Stone (Harry Potter, #1)" del género: ['Fantasy', 'Fiction', 'Young Adult', 'Magic', 'Childrens', 'Middle Grade', ...
   🎯 Libro encontrado - metadata extraída automáticamente


3. ORIGINAL: "similar a 'The Hunger Games'"
   Pertinente: ✅
   MEJORADO: Recomendar libros similares a "The Hunger Games (The Hunger Games, #1)" del género: ['Young Adult', 'Fiction', 'Dystopia', 'Fantasy', 'Science Fiction', 'Romance', 'Adventure'] con...
   🎯 Libro encont

INFO: ✓ Libro encontrado: To Kill a Mockingbird
INFO: ✓ Prompt mejorado generado
INFO: ✓ Prompt mejorado generado


   MEJORADO: Recomendar libros similares a "To Kill a Mockingbird" del género: ['Classics', 'Fiction', 'Historical Fiction', 'School', 'Literature', 'Young Adult', 'Historical'] con rating míni...
   🎯 Libro encontrado - metadata extraída automáticamente



## 10. Estadísticas del Dataset (Opcional)

Explora el dataset para entender mejor qué filtros puedes usar.

In [ ]:
if df is not None:
    print("\n" + "="*70)
    print("ESTADÍSTICAS DEL DATASET")
    print("="*70)
    
    print(f"\n📊 Información General:")
    print(f"   Total de libros: {len(df):,}")
    print(f"   Columnas: {len(df.columns)}")
    
    if 'Avg_Rating' in df.columns:
        print(f"\n⭐ Ratings:")
        print(f"   Promedio: {df['Avg_Rating'].mean():.2f}")
        print(f"   Mínimo: {df['Avg_Rating'].min():.2f}")
        print(f"   Máximo: {df['Avg_Rating'].max():.2f}")
        print(f"   Mediana: {df['Avg_Rating'].median():.2f}")
    
    if 'Num_Ratings' in df.columns:
        # Convertir a numérico si es necesario
        num_ratings = pd.to_numeric(df['Num_Ratings'], errors='coerce')
        print(f"\n👥 Número de Valoraciones:")
        print(f"   Total: {num_ratings.sum():,.0f}")
        print(f"   Promedio por libro: {num_ratings.mean():.0f}")
        print(f"   Máximo: {num_ratings.max():,.0f}")
    
    print(f"\n📚 Top 5 Libros Más Valorados:")
    if 'Num_Ratings' in df.columns:
        # Asegurar que Num_Ratings sea numérico para ordenar
        df_sorted = df.copy()
        df_sorted['Num_Ratings_numeric'] = pd.to_numeric(df_sorted['Num_Ratings'], errors='coerce')
        top = df_sorted.nlargest(5, 'Num_Ratings_numeric')[['Book', 'Author', 'Avg_Rating', 'Num_Ratings']]
        for idx, row in top.iterrows():
            print(f"   • {row['Book'][:50]}")
            num_votes = pd.to_numeric(row['Num_Ratings'], errors='coerce')
            if pd.notna(num_votes):
                print(f"     {row['Author']} - ⭐{row['Avg_Rating']} ({num_votes:,.0f} votos)")
            else:
                print(f"     {row['Author']} - ⭐{row['Avg_Rating']} ({row['Num_Ratings']} votos)")
    
    print(f"\n{'='*70}\n")
else:
    print("⚠️ Dataset no cargado - no se pueden mostrar estadísticas")



ESTADÍSTICAS DEL DATASET

📊 Información General:
   Total de libros: 10,000
   Columnas: 8

⭐ Ratings:
   Promedio: 4.07
   Mínimo: 0.00
   Máximo: 5.00
   Mediana: 4.08

👥 Número de Valoraciones:
   Total: 453,789
   Promedio por libro: 164
   Máximo: 999

📚 Top 5 Libros Más Valorados:
   • Hometown Girl After All (Hometown, #2)
     Kirsten Fullmer - ⭐4.32 (999 votos)
   • Hometown Girl After All (Hometown, #2)
     Kirsten Fullmer - ⭐4.32 (999 votos)
   • Belonging (Temptation, #2)
     Karen Ann Hopkins - ⭐3.98 (998 votos)
   • تشي
     أحمد خالد توفيق - ⭐4.0 (998 votos)
   • Living The Best Day Ever
     Hendri Coetzee - ⭐4.34 (997 votos)


   Mediana: 4.08

👥 Número de Valoraciones:
   Total: 453,789
   Promedio por libro: 164
   Máximo: 999

📚 Top 5 Libros Más Valorados:
   • Hometown Girl After All (Hometown, #2)
     Kirsten Fullmer - ⭐4.32 (999 votos)
   • Hometown Girl After All (Hometown, #2)
     Kirsten Fullmer - ⭐4.32 (999 votos)
   • Belonging (Temptation, #2)
     Kar